# Testing out ways to parse gpt chat responses

In [15]:
example_response = f"""
Main Points:
- The TPS reports need to be completed
- Everyone is expected to attend the dance party

Action Items:
- Mark will complete the TPS reports
- A calendar invite will be made for completing the TPS reports

Recent Summary:
- Mark will complete the TPS reports.
"""

In [7]:
def split(txt, seps):
    default_sep = seps[0]

    # we skip seps[0] because that's the default separator
    for sep in seps[1:]:
        txt = txt.replace(sep, default_sep)
    return [i.strip() for i in txt.split(default_sep)]

resp_groups = split(example_response, ['Action Items:', 'Main Points:', 'Recent Summary:'])

# Note: resp_groups[0] = ''
action_items = resp_groups[1]
main_points = resp_groups[2]
summary = resp_groups[3]

print(action_items.split('\n'))
print(main_points.split('\n'))

['- The TPS reports need to be completed', '- Everyone is expected to attend the dance party']
['- Mark will complete the TPS reports', '- A calendar invite will be made for completing the TPS reports']


# Testing out how to search around in drive

In [2]:
import os

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

credential_json = 'app_credential.json'
SCOPES = ['https://www.googleapis.com/auth/documents.readonly',
          'https://www.googleapis.com/auth/drive.readonly']


In [3]:

def get_credentials(force=False):
    """Gets valid user credentials from storage.

    If nothing has been stored, or if the stored credentials are invalid,
    the OAuth 2.0 flow is completed to obtain the new credentials.

    Returns:
        Credentials, the obtained credential.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if not force and os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credential_json, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    return creds

In [29]:
creds = get_credentials()


def get_most_recent_transcript_id(creds):
    drive_service = build('drive', 'v3', credentials=creds)

    # first find the transcript folder
    results = drive_service.files().list(
        q="mimeType = 'application/vnd.google-apps.folder' and fullText contains 'Meet Transcript'",
        spaces='drive',
        pageSize=5,
        fields="nextPageToken, files(id, name, parents)").execute()
    folders = results.get('files', [])

    if folders is None:
        raise Exception("Meet Transcript folder not found within google drive! You need to start a Google Meet and a transcript!\nDownload transcript generation app here: https://chrome.google.com/webstore/detail/meet-transcript/jkdogkallbmmdhpdjdpmoejkehfeefnb?hl=en")
        
    assert len(folders) == 1
    # print("Folders:")
    # print(folders[0])

    # Use the transcript folder to query for the most recent document 
    query = "mimeType='application/vnd.google-apps.document' and '" + str(folders[0]['id']) + "' in parents"
    results = drive_service.files().list(
        q=query, 
        spaces='drive',
        pageSize=5, 
        fields="nextPageToken, files(id, parents, name)").execute()
    files = results.get('files', [])

    # NOTE: results are sorted by recent first.
    if len(files) > 0:
        print('Selected transcript file {}'.format(files[0]['name']))
        return files[0]['id']
    
    return None

print(get_most_recent_transcript_id(creds))

Folders:
{'parents': ['0AFuZxqT9lSZOUk9PVA'], 'id': '1O47S3z0aliFBFnugkF4-_t-oNYMv2Yz7', 'name': 'Meet Transcript'}
mimeType='application/vnd.google-apps.document' and '1O47S3z0aliFBFnugkF4-_t-oNYMv2Yz7' in parents
Target File:
{'parents': ['1O47S3z0aliFBFnugkF4-_t-oNYMv2Yz7'], 'id': '1qPJ_sb9W0VhdpsFXhOQP9BFbLVK4dIpvr8alG_mHblU', 'name': '2022/12/10 18:45 - Sat Dec 10 6:45 pm'}
